---
### RabbitMQ 서버 및 MariaDB에 연결하는 코드
---

In [25]:
# 모듈 임포트
import pika
import mysql.connector

import sys
import time

In [16]:
# 콜백 함수
def callback(ch, method, properties, body):
    message = body.decode('utf-8')
    print(" [x] Received %r" % message)
    
    # MariaDB에 연결
    conn = mysql.connector.connect(
        host="34.64.63.23",
        user="big17",
        password="1234",
        database="trading"
    )
    
    # 커서 생성
    cursor = conn.cursor()
    
    # 메시지를 DB에 저장
    sql = ""
    val = (message, )
    # SQL문 실행
    cursor.execute(message)
    
    # 커서 커밋 및 확인
    conn.commit()
    print(cursor.rowcount, 'record(s) inserted')
    
    # 연결 종료
    cursor.close()
    conn.close()

In [18]:
# RabbitMQ 설정
credentials = pika.PlainCredentials('big17', '1234')
parameters = pika.ConnectionParameters(
      host='34.64.63.23'
    , port=5672
    , virtual_host='/'
    , credentials=credentials
)
connection = pika.BlockingConnection(parameters)

channel = connection.channel()
channel.queue_declare(queue='test')

# Queue에서 메시지를 받을 준비
channel.basic_consume(
      queue='test'
    , on_message_callback=callback
    , auto_ack=True
)
print(" [*] Waiting for messages. To exit press CTRL+C")
channel.start_consuming()

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=test'])>"])>

---
### Producer 코드
---
작업을 생성하고 큐에 추가하는 코드

In [23]:
# RabbitMQ 서버에 연결
credentials = pika.PlainCredentials('big17', '1234')
parameters = pika.ConnectionParameters(
      host='34.64.63.23'
    , port=5672
    , virtual_host='/'
    , credentials=credentials
)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

# 'job_queue'(예시)라는 이름의 큐를 선언
channel.queue_declare(queue='job_queue', durable=True)

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=job_queue'])>"])>

In [26]:
# 큐에 작업 추가
message = ' '.join(sys.argv[1:]) # 메시지 내용 설정
channel.basic_publish(
      exchange='' # 교환명(도큐 읽어보니까 비워도 됩니다. -승현)
    , routing_key='job_queue' # 라우팅 키(QUEUE_NAME)
    , body=message # 큐에 싣는 메시지
    , properties=pika.BasicProperties(delivery_mode=2) # 메시지를 영구적으로 만듭니다.
)
print(" [x] Sent %r" % message)

# 연결 종료
connection.close()

 [x] Sent '--f=c:\\Users\\SHLee\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-319963SiIZll0qGeJ.json'


---
### Consumer 코드
---
큐에서 작업을 받아 처리하는 코드

In [27]:
# 콜백 함수
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body.decode('utf-8'))
    time.sleep(body.count(b'.')) # 메시지 내용에 따라 처리 시간을 달리함
    print(" [x] Done")
    ch.basic.ack(delivery_tag=method.delivery_tag)

In [ ]:
# RabbitMQ 서버에 연결
credentials = pika.PlainCredentials('big17', '1234')
parameters = pika.ConnectionParameters(
      host='34.64.63.23'
    , port=5672
    , virtual_host='/'
    , credentials=credentials
)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

# 'job_queue'(예시)라는 이름의 큐를 선언
channel.queue_declare(queue='job_queue', durable=True)

# 한번의 하나의 메시지만 처리하도록 요청
channel.basic_qos(prefetch_count=1)

# 메시지 처리
channel.basic_consume(
    queue='job_queue'
  , on_message_callback=callback
  , auto_ack=True
)

# 처리 준비 완료 메시지 출력
# 놔두면 무한 대기합니다. 테스트만 할거면 적당한 시점에 컨트롤+C 해서 강제종료하세요.
print(" [*] Waiting for messages. To exit press CTRL+C")
channel.start_consuming()